# A script to get current fires per TFR from Ororatech API
and to find exceeding ones

In [1]:
import requests
import json
import geopandas as gpd
import pandas as pd
import sys

## Get the current fires

In [2]:
#provide credentials for API access, obtain from Ororatech and insert here 
APIkey = 'someSuperLongApiKey'

#### A single hotspot is only contained in the API response, if its center is within the bounding coordinates. So it is no clean solution to search for fires using TFR bounding boxes. Instead, buffered fire clusters and TFR polygons need to get joined first. Then, the boundary box of the fire cluster can be used to spatially limit the area, where a time based search for hotspots being active within TFR runtime can be performed to detect runaway fires.
As it turned out that ARTCCs issue TFRs beyond their FIR´s boundary, the entire fire cluster dataset has to be joined each time and will be read therefor. 

In [3]:
##specify file location and name
#3 letter location indicator, reused within all created files
#possible values: ZAB ZDV ZFW ZLA ZLC ZMP ZOA ZSE; not ZKC and ZHU having no wildfire TFRs
tfr =r"ZDV"
#paths and filenames
fire_path = r"D:\UNIGIS\MASTER\DownloadedData\WFS\\"
tfr_path = r"D:\UNIGIS\MASTER\Scripts\\"
out_path = r"D:\UNIGIS\MASTER\Scripts\Exceeding\\"

fire_file = "wfs-area-export_FIRs_Boundary_08-102021_con_pt5.geojson"
tfr_file = tfr + r"_fire_TFRs.geojson"

#read geojson files
gdf_fires = gpd.read_file(fire_path + fire_file)
gdf_tfrs = gpd.read_file(tfr_path + tfr_file)

#size to buffer fires in m; 1609.344 m = 1 SM, 4828.032 m = 3 SM
buf_size = 4828.032

#output concatenates from these strings as well. Buffer size must be a string for that as well
str_buf_size = str(int(buf_size/1609.344))

#output filename (if any):
outfile = out_path+tfr+'_'+str_buf_size+'SM_runaway_fires.geojson'

#logfile filename (to append some row/feature counts)
logfilename = out_path+'Fires_from_API_log.txt'

Number of evaluated TFRs per FIR, ZDV: 28

In [4]:
print(len(gdf_tfrs.index))

28


In [5]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of evaluated TFRs for '+tfr+' '+str_buf_size+' SM: '+str(len(gdf_tfrs.index))+'\n' )

### Buffer fire clusters

Without any temporal relation yet, the same fire may be tied to multiple TFRs. For fires close to each other, issuing one large contiguous TFR is allowed, so the same TFR might contain multiple fires as well. So the resulting geodataframe can contain a multiple of rows compared to the origin.
As the fire clusters used so far represent the largest extent of the fires within the observed time period (August to October 2021), those not leaving their TFR (after the fire cluster got buffered) can be omitted here. This is also done to limit the amount of API requests. But those fires (plus buffer) crossing TFR boundaries (=overlap in shapely terms) do need a closer look considering time.

Now buffer fire clusters by specified amount of statute miles to get prepared for the overlap. 

In [6]:
#prepare buffer with a metric CRS
gdf_fires_buffered = gdf_fires.copy()
gdf_fires_buffered = gdf_fires_buffered.to_crs("EPSG:2163")

#perform buffer by amount of statute miles (SM), 1 SM = 1609.344 m
gdf_fires_buffered["geometry"] = gdf_fires_buffered.buffer(buf_size, resolution=16)

#turn CRS back
gdf_fires_buffered = gdf_fires_buffered.to_crs("EPSG:4326")


In [7]:
gdf_fires_buffered.head(2)

,id,age,area,centroid,num_fires,confidence,newest_detection,oldest_detection,newest_acquisition,oldest_acquisition,geometry
0,19778761,131964,1.697506e+07,"{'latitude': 29.051065, 'longitude': -97.286025}",47,0.8,2021-08-01T11:07:57+00:00,2021-07-31T23:25:52+00:00,2021-08-01T08:36:21+00:00,2021-07-31T23:15:05+00:00,"POLYGON ((-97.29353 29.11033, -97.28962 29.111..."
1,19775166,131967,9.360254e+06,"{'latitude': 43.104132, 'longitude': -102.572029}",45,1.0,2021-08-01T11:04:44+00:00,2021-08-01T03:25:31+00:00,2021-08-01T08:32:35+00:00,2021-07-31T19:25:18+00:00,"POLYGON ((-102.63812 43.11719, -102.63716 43.1..."


### Join buffered fire clusters and TFRs

Geopandas sjoin is "one to many" automatically. Inner join is needed to limit fire clusters to those actually fulfilling the predicate.  

Just to look for intersect is expected to deliver way too many fire clusters (even with unbuffered ones).

In [8]:
#perform join of gdfs
#gdf_intersect = gpd.sjoin(gdf_fires, gdf_tfrs, how='inner', predicate='intersects')
#gdf_intersect

In [9]:
#If it is ever needed to join a join result again, index columns need to get renamend
#gdf_intersect.rename(columns = {'index_right':'old_index_right'}, inplace = True)

In [10]:
gdf_runfires_bybuff = gpd.sjoin(gdf_fires_buffered, gdf_tfrs, how='inner', predicate='overlaps')

This is one possible "exit point": If nothing overlaps, then the buffered fires are contained within the TFRs and the resulting geodataframe has 0 rows. Exit is then performed after the follwoing 2 log entries.

Within the observed data, this is the case for: ZAB

However, there is an exception for an extreme case, if the fire plus buffer has already spread over the entire TFR, this has to be checked:

In [11]:
#check for buffered fire already covering entire TFR
gdf_coveringfires_bybuff = gpd.sjoin(gdf_fires_buffered, gdf_tfrs, how='inner', predicate='contains')

In [12]:
gdf_coveringfires_bybuff.head(2)

,id,age,area,centroid,num_fires,confidence,newest_detection,oldest_detection,newest_acquisition,oldest_acquisition,...,Source,Location,NOTAM #,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m
862,20866464,87258,2.916087e+07,"{'latitude': 40.1618, 'longitude': -106.237305}",587,1.0,2021-09-01T12:22:13+00:00,2021-08-29T22:24:39+00:00,2021-09-01T09:42:27+00:00,2021-08-29T19:31:23+00:00,...,ZDV_2021-08-30,ZDV,1/8134,08/30/2021 0201,08/30/2021 1400,09/07/2021 1405,10/29/2021 0200,!FDC 1/8134 ZDV CO..AIRSPACE 8NM NE OF KREMLIN...,3.0,5556.0


In [13]:
#concatenate dataframes if buffered fires contain an entire TFR
if len(gdf_coveringfires_bybuff.index)>0:
    frames = [gdf_runfires_bybuff, gdf_coveringfires_bybuff]
    gdf_runfires_bybuff = gpd.GeoDataFrame(pd.concat(frames, sort=False))

    

In [14]:
# change the global options that Geopandas inherits from if more rows/columns shall be displayed
# pd.set_option('display.max_columns',None)
gdf_runfires_bybuff.head(2)

,id,age,area,centroid,num_fires,confidence,newest_detection,oldest_detection,newest_acquisition,oldest_acquisition,...,Source,Location,NOTAM #,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m
257,19952554,121182,4.859015e+07,"{'latitude': 41.527944, 'longitude': -103.350363}",635,1.0,2021-08-09T18:11:45+00:00,2021-08-06T04:06:48+00:00,2021-08-09T02:55:05+00:00,2021-08-06T02:55:05+00:00,...,ZDV_2021-08-06,ZDV,1/2974,08/06/2021 1847,08/06/2021 1830,None,08/07/2021 0400,!FDC 1/2974 ZDV CANCELLED BY FDC 1/3037 ON 08/...,7.0,12964.0
257,19952554,121182,4.859015e+07,"{'latitude': 41.527944, 'longitude': -103.350363}",635,1.0,2021-08-09T18:11:45+00:00,2021-08-06T04:06:48+00:00,2021-08-09T02:55:05+00:00,2021-08-06T02:55:05+00:00,...,ZDV_2021-08-07,ZDV,1/3125,08/07/2021 1436,08/07/2021 1450,08/07/2021 2018,08/08/2021 0300,!FDC 1/3125 ZDV NE..AIRSPACE 24NM SSE SCOTTSBL...,7.0,12964.0


Number of cases where fire leaves TFR

In [15]:
print(len(gdf_runfires_bybuff.index))

11


ZDV (3 SM): 11

ZDV (1 SM): 5

In [16]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of potential cases where fire leaves TFR for '+tfr+' '+str_buf_size+' SM: '+str(len(gdf_runfires_bybuff.index))+'\n' )

### Tasks to fill payload to get current fires:

To create a payload per fire per TFR, each time
A) bounding box coordinates and

B) a time period represented as (end) 'date' and duration in 'minutes' are needed.

### A) Bounding Box

The buffered fire clusters overlapping any TFR are potentially those causing a safety threat. The following "selection" shows, how many there are.

In [17]:
# Select fire clusters that overlapped while being buffered to get a fire(row) count to mention.
options = gdf_runfires_bybuff["id"]
gdf_fires_tofetch = gdf_fires[gdf_fires["id"].isin(options)]

In [18]:
print(len(gdf_fires_tofetch.index))

7


Number of fire events leaving TFR

ZDV (3 SM): 7

ZDV (1 SM): 1

In [19]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of potential fire events leaving TFR for '+tfr+' '+str_buf_size+' SM: '+str(len(gdf_fires_tofetch.index))+'\n' )

In [20]:
# exit script with an empty result file if intersect geodataframe has 0 rows
if len(gdf_runfires_bybuff.index) ==0:
    sys.exit(0)

For simplification, the work is continued with the buffered dataset. Bounding box coordinates are added to the geodataframe.

In [21]:
# Geopands´ .bounds delivers coordinates of the boundary boxes
gdf_bbox = gdf_runfires_bybuff.bounds

In [22]:
gdf_bbox.head(2)

,minx,miny,maxx,maxy
257,-103.472792,41.444095,-103.24504,41.60568
257,-103.472792,41.444095,-103.24504,41.60568


In [23]:
#just use pd.concat / axis=1 to append boundary box coordinates
gdf_runfires_bybuff = pd.concat([gdf_runfires_bybuff, gdf_bbox], axis=1)

In [24]:
#check attached columns, if needed
gdf_runfires_bybuff.head(2)

,id,age,area,centroid,num_fires,confidence,newest_detection,oldest_detection,newest_acquisition,oldest_acquisition,...,Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m,minx,miny,maxx,maxy
257,19952554,121182,4.859015e+07,"{'latitude': 41.527944, 'longitude': -103.350363}",635,1.0,2021-08-09T18:11:45+00:00,2021-08-06T04:06:48+00:00,2021-08-09T02:55:05+00:00,2021-08-06T02:55:05+00:00,...,08/06/2021 1830,None,08/07/2021 0400,!FDC 1/2974 ZDV CANCELLED BY FDC 1/3037 ON 08/...,7.0,12964.0,-103.472792,41.444095,-103.24504,41.60568
257,19952554,121182,4.859015e+07,"{'latitude': 41.527944, 'longitude': -103.350363}",635,1.0,2021-08-09T18:11:45+00:00,2021-08-06T04:06:48+00:00,2021-08-09T02:55:05+00:00,2021-08-06T02:55:05+00:00,...,08/07/2021 1450,08/07/2021 2018,08/08/2021 0300,!FDC 1/3125 ZDV NE..AIRSPACE 24NM SSE SCOTTSBL...,7.0,12964.0,-103.472792,41.444095,-103.24504,41.60568


### B) (Part 1) Get minutes for the API request

Getting the time values for the request cannot be done on column level as the Cancel Date (UTC) may be empty or even be before the Effective Date (UTC) if there is no longer a threat or a flight planned.

In [25]:
#get minutes value for the API request
def get_minutes(row):
    startdate = row["Effective Date (UTC)"]
    
    enddate= row["Cancel Date (UTC)"]
    if enddate == None:
        enddate= row["Expiration Date (UTC)"]
    
    duration = pd.to_datetime(enddate, errors='coerce') - pd.to_datetime(startdate, errors='coerce')
        
    minutes = duration.total_seconds()/60
    
    minutes = int(minutes)
    
    return minutes
gdf_runfires_bybuff["Minutes"]=gdf_runfires_bybuff.apply(get_minutes,axis=1)

In [26]:
#column-wise calculation if it was possible
# gdf_runfires_bybuff["restr_duration"] = pd.to_datetime(gdf_runfires_bybuff["Cancel Date (UTC)"], errors='coerce') - pd.to_datetime(gdf_runfires_bybuff["Effective Date (UTC)"], errors='coerce')
# gdf_runfires_bybuff["Minutes"] = gdf_runfires_bybuff["restr_duration"].dt.total_seconds().div(60)

In [27]:
gdf_runfires_bybuff["Minutes"].head(2)

257    570
257    328
Name: Minutes, dtype: int64

For the API, 0 or a negative value is invalid: {'message': 'Invalid range for minutes parameter'} Thus, the related rows need to become removed

In [28]:
#keep only rows where "Minutes" >= 0
gdf_runfires_bybuff = gdf_runfires_bybuff[gdf_runfires_bybuff["Minutes"]>=0]

### B) (Part 2) Get date(time) for the API request

The date from when the API goes back needs to get acquired as follows: 'date': '2021-08-16-0200'

In [29]:
#get date for the API request
def get_apidate(row):
    
    enddate= row["Cancel Date (UTC)"]
    if enddate == None:
        enddate= row["Expiration Date (UTC)"]
        #if both enddate columns become read, a SettingWithCopyWarning occurs, which is ok
    
    #endate is 'MM/DD/YYYY hhmm' format (s string)
    #turn to  'YYYY-MM-DD-hhmm' format for API. Mind strftime() Directives
    enddate = pd.to_datetime(enddate,errors='coerce')
    enddate = enddate.strftime('%Y-%m-%d-%H%M')
    
    apidate = enddate
    
    return apidate
gdf_runfires_bybuff["APIdate"]=gdf_runfires_bybuff.apply(get_apidate,axis=1)

In [30]:
gdf_runfires_bybuff["APIdate"].head(2)

257    2021-08-07-0400
257    2021-08-07-2018
Name: APIdate, dtype: object

### Perform the API request

A minimum confidence of 0.5 is recommended by Ororatech for analysis of historical data. So, 'confidence' must be set to 0.4 as the API looks for everything ABOVE.

In [31]:
def get_clusterPerAPI(row):
    #collect payload content per row
    xmin_pl = str(row["minx"])
    ymin_pl = str(row["miny"])
    xmax_pl = str(row["maxx"])
    ymax_pl = str(row["maxy"])
    minute_pl = str(row["Minutes"])
    date_pl = str(row["APIdate"])
    
    payload = {'xmin': xmin_pl,
                   'ymin': ymin_pl,
                   'xmax': xmax_pl,
                   'ymax': ymax_pl,
                   'minutes': minute_pl,
                   'date': date_pl,
                   'confidence': '0.4',
                   'select': ['oldest_detection,oldest_acquisition,types'] ,
                   'token': APIkey}
    
    #the request:
    response = requests.get('https://app.ororatech.com/v1/clusters/',params=payload)
    
    #test request:
    #testpayload = {'xmin': '-117.86', 'ymin': '47.88', 'xmax': '-117.53', 'ymax': '48.00', 'minutes': '360', 'date': '2021-08-16-0200','confidence': '0.5', 'token': APIkey}
    #response = requests.get('https://app.ororatech.com/v1/clusters/',params=testpayload)
    
    data = response.json()
    #if data is not None does not help in case of an "empty" json
    #in that case, response.json() = {'type': 'FeatureCollection', 'features': None}
    #would lead to an error, trying to create agdf from features
    if data != {'type': 'FeatureCollection', 'features': None}:
        #columns=['geometry', 'id', 'num_fires']
        gdf_local = gpd.GeoDataFrame.from_features(data)
    else:
        #else prepare an empty gdf for return
        gdf_local = gpd.GeoDataFrame()
    
    return gdf_local

series_of_gdfs = gdf_runfires_bybuff.apply(get_clusterPerAPI,axis=1)
list_of_gdfs= series_of_gdfs.tolist()

#concat returned gdfs; empty ones are not cosidered by default
gdf_current = gpd.GeoDataFrame(pd.concat(list_of_gdfs, ignore_index=True))

#gdf_current is a geodataframe, but crs has still to be specified
gdf_current = gdf_current.set_crs("EPSG:4326", allow_override=True)


In [32]:
gdf_current.head(2)

,geometry,id,types,num_fires,oldest_detection,oldest_acquisition
0,"POLYGON ((-103.39699 41.50449, -103.39690 41.5...",19952554,[0],635,2021-08-06T22:53:25+00:00,2021-08-06T02:55:05+00:00
1,"POLYGON ((-103.38383 41.50191, -103.38270 41.5...",19952554,[0],635,2021-08-07T18:53:00+00:00,2021-08-06T02:55:05+00:00


In [33]:
# field "types" contains list for Python. They need to be converted to strings,
# otherwise .duplicated() or turning to GeoJSON would not work!
gdf_current["types"] = gdf_current["types"].astype('string')
# remove [] to avoid confusion when this is read from GeoJSON again
gdf_current["types"] = gdf_current["types"].str.removeprefix("[")
gdf_current["types"] = gdf_current["types"].str.removesuffix("]")


In [34]:
gdf_current["types"].head(2)

0    0
1    0
Name: types, dtype: string

### Buffer current fires

The API request results get buffered by specified (variable buf_size) amount of statute miles.

In [35]:
#prepare buffer with a metric CRS
gdf_current_buffered = gdf_current.copy()
gdf_current_buffered = gdf_current_buffered.to_crs("EPSG:2163")

#perform buffer by specified amount of statute miles (SM)
gdf_current_buffered["geometry"] = gdf_current_buffered.buffer(buf_size, resolution=16)

#turn CRS back
gdf_current_buffered = gdf_current_buffered.to_crs("EPSG:4326")

If there is an overlap between TFRs and API request results, then those TFRs can be considered inappropriate from an aerial firefighting perspective.

In [36]:
gdf_result = gpd.sjoin(gdf_current_buffered, gdf_tfrs, how='inner', predicate='overlaps')

In [37]:
gdf_result.head(2)

,geometry,id,types,num_fires,oldest_detection,oldest_acquisition,index_right,Source,Location,NOTAM #,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m
3,"POLYGON ((-107.11377 36.94489, -107.11362 36.9...",20021038,0,939,2021-08-09T23:13:52+00:00,2021-08-07T17:52:20+00:00,10,ZDV_2021-08-08,ZDV,1/3260,08/08/2021 1648,08/08/2021 1700,08/09/2021 0311,08/10/2021 0300,!FDC 1/3260 ZDV NM..AIRSPACE 47NM NE OF BLOOMF...,5.0,9260.0
3,"POLYGON ((-107.11377 36.94489, -107.11362 36.9...",20021038,0,939,2021-08-09T23:13:52+00:00,2021-08-07T17:52:20+00:00,13,ZDV_2021-08-09,ZDV,1/3434,08/09/2021 0312,08/09/2021 1300,08/12/2021 1402,09/09/2021 0300,!FDC 1/3434 ZDV NM..AIRSPACE 47NM NE OF BLOOMF...,5.0,9260.0


A TFR is also inappropriate, if it is entirely inside a fire, so the according check from above is reused

In [38]:
#check for buffered fire already covering entire TFR
gdf_currentcovering_bybuff = gpd.sjoin(gdf_current_buffered, gdf_tfrs, how='inner', predicate='contains')

In [39]:
gdf_currentcovering_bybuff.head(2)

,geometry,id,types,num_fires,oldest_detection,oldest_acquisition,index_right,Source,Location,NOTAM #,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m
6,"POLYGON ((-106.32305 40.16465, -106.32301 40.1...",20866464,0,587,2021-08-30T18:51:21+00:00,2021-08-29T19:31:23+00:00,18,ZDV_2021-08-30,ZDV,1/8134,08/30/2021 0201,08/30/2021 1400,09/07/2021 1405,10/29/2021 0200,!FDC 1/8134 ZDV CO..AIRSPACE 8NM NE OF KREMLIN...,3.0,5556.0


In [40]:
#concatenate dataframes if buffered fires contain an entire TFR
if len(gdf_currentcovering_bybuff.index)>0:
    frames = [gdf_result, gdf_currentcovering_bybuff]
    gdf_result = gpd.GeoDataFrame(pd.concat(frames, sort=False))

In [41]:
print(len(gdf_result.index))

5


Number of results: ZDV (3 SM): 11, ZDV (1 SM): 1

In [42]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number results for '+tfr+' '+str_buf_size+' SM: '+str(len(gdf_result.index))+'\n' )

ATTENTION: As joined once again with all TFRs in gdf_tfrs, those TFR cancelled before being effective will be included here! So get_minutes function is applied once more to recognize those. The Number of occurrences gets printed below. To obtain the true count (without duplicates), this must be applied on the input geodataframe gdf_tfrs

In [43]:
#apply function get_minutes from above again
gdf_tfrs["Duration_Minutes"]=gdf_tfrs.apply(get_minutes,axis=1)

In [44]:
print(len(gdf_tfrs[gdf_tfrs["Duration_Minutes"]<=0].index))

1


Number of TFR(s) cancelled before becoming effective:
ZDV (3 SM): 1
ZDV (1 SM): 0

In [45]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of TFR(s) cancelled before becoming effective for '+tfr+' '+str_buf_size+' SM: '+str(len(gdf_tfrs[gdf_tfrs["Duration_Minutes"]<=0].index))+'\n' )

In [46]:
gdf_result.head(2)

,geometry,id,types,num_fires,oldest_detection,oldest_acquisition,index_right,Source,Location,NOTAM #,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject,Radius,Radius_m
3,"POLYGON ((-107.11377 36.94489, -107.11362 36.9...",20021038,0,939,2021-08-09T23:13:52+00:00,2021-08-07T17:52:20+00:00,10,ZDV_2021-08-08,ZDV,1/3260,08/08/2021 1648,08/08/2021 1700,08/09/2021 0311,08/10/2021 0300,!FDC 1/3260 ZDV NM..AIRSPACE 47NM NE OF BLOOMF...,5.0,9260.0
3,"POLYGON ((-107.11377 36.94489, -107.11362 36.9...",20021038,0,939,2021-08-09T23:13:52+00:00,2021-08-07T17:52:20+00:00,13,ZDV_2021-08-09,ZDV,1/3434,08/09/2021 0312,08/09/2021 1300,08/12/2021 1402,09/09/2021 0300,!FDC 1/3434 ZDV NM..AIRSPACE 47NM NE OF BLOOMF...,5.0,9260.0


Result may contain duplicates, due to multi join and fires overlapping TFRs more than one time. Resulting log gets a count of entirely duplicate rows, the 'Number of TFRs where a fire leaves the TFR' and the 'Number of fire events leaving a TFR'

In [47]:
#count duplicates
gdf_result.duplicated().sum()

0

In [48]:
#count non-duplicates
(~gdf_result.duplicated()).sum()

5

In [49]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of results without duplicates for '+tfr+' '+str_buf_size+' SM: '+str((~gdf_result.duplicated()).sum())+'\n' )

In [50]:
# count non-duplicates of TFRs
(~gdf_result.duplicated(["NOTAM #"])).sum()

5

In [51]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of TFRs where a fire leaves the TFR for '+tfr+' '+str_buf_size+' SM: '+str((~gdf_result.duplicated(["NOTAM #"])).sum())+'\n' )

In [52]:
# count non-duplicates of fires
(~gdf_result.duplicated(["id"])).sum()

3

In [53]:
# Appending to logfile
with open(logfilename, 'a') as logfile:
    logfile.write('Number of fire events leaving a TFR for '+tfr+' '+str_buf_size+' SM: '+str((~gdf_result.duplicated(["id"])).sum())+'\n'+'\n' )

In [54]:
# Considering certain columns for dropping duplicates, if wanted
#gdf_result.drop_duplicates(subset=['id', 'NOTAM #'])

# Drop entire row duplicates, if wanted
#gdf_result.drop_duplicates()

Finally, compose an output file for those fires that had inappropriate TFRs, IF any

In [55]:
# GeoJSON output
if len(gdf_result.index)>0:
    gdf_result.to_file(filename= outfile, driver='GeoJSON')

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


This Notebook ends here. Follow up analysis is performed Get_Events_from_Fires_from_API to avoid rerunning API requests when this is not necessary.